## This nothebook shows how to compute the energy consumption for a given quantized network. 

In [1]:
import tensorflow as tf
import pickle
import numpy as np
import imageio
import json
from scipy import misc
import timeit
import math
import os
from time_space_eval_pathonet import make_submodel, \
        extract_conv_info, extract_compressed_weights, \
        read_labels, space_calc, product_time, \
        residualDilatedInceptionModule_iMap, PathoNet_iMaps, \
        utils_for_Patho_iMap

2023-05-29 17:14:16.226857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 17:14:16.226872: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Avoid printing INFO, WARNING, and ERROR messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
#Activate GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2023-05-29 17:14:19.309212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 17:14:19.309420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 17:14:19.325341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-05-29 17:14:19.325402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2023-05-29 17:14:19.325452: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

In [4]:
# Load original model
model = tf.keras.models.load_model("./original_nets/PathoNet.hdf5")

2023-05-29 17:14:21.394750: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Change this path to use a different compressed model

path_quant="./pathonet_compressed_models/experiments/UQ_k256/0-0-0-249-0.0001-0.0001-5-75-146.19026.h5" #CWS_k256/0-0-0-256-0.0001-1e-05-5-75-147.1456.h5"

In [6]:
# Load the considered "quantized" model and set weights in original model structure
with open(path_quant, "rb") as weights:
    lw = pickle.load(weights)
model.set_weights(lw)

In [7]:
#Load image for testing
imageShape = [1228, 1228, 3]
    
inputShape = [256, 256, 3]

### Following, the creation of the PathoNet iMap model and how to load it

In [8]:
from tensorflow.keras.layers import (Input,Add,add,concatenate,Activation,concatenate,
                        Concatenate,Dropout,BatchNormalization,Reshape,Permute,
                        Dense,UpSampling2D,Flatten,Lambda,Activation,Conv2D,
                        DepthwiseConv2D,ZeroPadding2D,GlobalAveragePooling2D,
                        MaxPooling2D,AveragePooling2D,LeakyReLU,Conv2DTranspose)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import get_source_inputs
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD, Adam

In [9]:
#Original Code of PathoNet

def residualDilatedInceptionModule(y, nb_channels, _strides=(1, 1),t="e"):
    if t=="d":
        y = Conv2D(nb_channels, kernel_size=(1, 1), strides=(1, 1),kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        y = Conv2D(nb_channels, kernel_size=(1, 1), strides=(1, 1),kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)


    A1 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
    A1 = BatchNormalization()(A1)
    A1 = LeakyReLU()(A1)
    A1 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(A1)
    A1 = BatchNormalization()(A1)
    A1 = LeakyReLU()(A1)


    A4 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4),  dilation_rate=4, padding='same', use_bias=False)(y)
    A4 = BatchNormalization()(A4)
    A4 = LeakyReLU()(A4)
    A4 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4),  dilation_rate=4, padding='same', use_bias=False)(A4)
    A4 = BatchNormalization()(A4)
    A4 = LeakyReLU()(A4)

    if (t=="e"):
        y=concatenate([y,y])
    y=add([A1,A4,y])
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)

    return y


def PathoNet(input_size = (256,256,3), classes=3, pretrained_weights = None):
    inputs = Input(input_size) 

    block1= Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(inputs)
    block1 = BatchNormalization()(block1)
    block1 = LeakyReLU()(block1)
    block1= Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block1)
    block1 = BatchNormalization()(block1)
    block1 = LeakyReLU()(block1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(block1)

    block2= residualDilatedInceptionModule(pool1,32,t="e")
    pool2 = MaxPooling2D(pool_size=(2, 2))(block2)

    block3= residualDilatedInceptionModule(pool2,64,t="e")
    pool3 = MaxPooling2D(pool_size=(2, 2))(block3)

    block4= residualDilatedInceptionModule(pool3,128,t="e")
    pool4 = MaxPooling2D(pool_size=(2, 2))(block4)
    drop4 = Dropout(0.1)(pool4)

    block5= residualDilatedInceptionModule(drop4,256,t="e")
    drop5 = Dropout(0.1)(block5)

    up6 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(drop5)),128,t="d")
    merge6 = concatenate([block4,up6], axis = 3)

    up7 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge6)),64,t="d")
    merge7 = concatenate([block3,up7], axis = 3)

    up8 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge7)),32,t="d")
    merge8 = concatenate([block2,up8], axis = 3)

    up9 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge8)),16,t="d")
    merge9 = concatenate([block1,up9], axis = 3)

    block9=Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(merge9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    block9=Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    block9=Conv2D(8, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    conv10 = Conv2D(classes, 1, activation = 'relu')(block9)

    model = tf.keras.models.Model(inputs = inputs, outputs = conv10)

    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    

    
    return model

In [10]:
#Reload weights and set on model created as new

with open(path_quant, "rb") as weights:
    lw = pickle.load(weights)

patho = PathoNet(input_size = (256,256,3), classes=3)
patho.set_weights(lw)

In [11]:
#Create PathoNet with iMap

res_weights, last_bias, indexes_weights, vect_centers = utils_for_Patho_iMap(patho)
patho_imap = PathoNet_iMaps(last_bias, indexes_weights, vect_centers, res_weights = res_weights)

In [12]:
img = imageio.imread("p9_0219_9.jpg")

/tmp/ipykernel_1124134/4223457834.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread("p9_0219_9.jpg")


In [18]:
img=np.expand_dims(np.array(misc.imresize(img,size=(inputShape[0], inputShape[1])))/255, axis=0)

## POWER CONSUMPTION

In [15]:
from codecarbon import EmissionsTracker

In [16]:
img1 = np.expand_dims(img, axis = 0)
img_tmp = img1.squeeze(axis=0)
pred = np.array([img_tmp]*500)

### POWER CONSUMPTION WITH COMPRESSED MODEL

In [21]:
with EmissionsTracker() as tracker:
    x = patho_imap.predict(pred, batch_size=50, verbose=0)

[codecarbon INFO @ 17:24:49] [setup] RAM Tracking...
[codecarbon INFO @ 17:24:49] [setup] GPU Tracking...
[codecarbon INFO @ 17:24:49] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:24:49] [setup] CPU Tracking...
[codecarbon WARNING @ 17:24:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:24:50] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750HF CPU @ 2.60GHz
[codecarbon INFO @ 17:24:50] >>> Tracker's metadata:
[codecarbon INFO @ 17:24:50]   Platform system: Linux-6.2.6-76060206-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 17:24:50]   Python version: 3.10.6
[codecarbon INFO @ 17:24:50]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 17:24:50]   Available RAM : 15.531 GB
[codecarbon INFO @ 17:24:50]   CPU count: 12
[codecarbon INFO @ 17:24:50]   CPU model: Intel(R) Core(TM) i7-9750HF CPU @ 2.60GHz
[codecarbon INFO @ 17:24:50]   GPU count: 1
[codecarbon INFO @ 17:24:50]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon 

### POWER CONSUMPTION WITH ORIGINAL MODEL

In [22]:
with EmissionsTracker() as tracker:
    x = patho.predict(pred, batch_size=50, verbose=0)

[codecarbon INFO @ 17:25:28] [setup] RAM Tracking...
[codecarbon INFO @ 17:25:28] [setup] GPU Tracking...
[codecarbon INFO @ 17:25:28] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:25:28] [setup] CPU Tracking...
[codecarbon WARNING @ 17:25:28] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 17:25:30] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-9750HF CPU @ 2.60GHz
[codecarbon INFO @ 17:25:30] >>> Tracker's metadata:
[codecarbon INFO @ 17:25:30]   Platform system: Linux-6.2.6-76060206-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 17:25:30]   Python version: 3.10.6
[codecarbon INFO @ 17:25:30]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 17:25:30]   Available RAM : 15.531 GB
[codecarbon INFO @ 17:25:30]   CPU count: 12
[codecarbon INFO @ 17:25:30]   CPU model: Intel(R) Core(TM) i7-9750HF CPU @ 2.60GHz
[codecarbon INFO @ 17:25:30]   GPU count: 1
[codecarbon INFO @ 17:25:30]   GPU model: 1 x NVIDIA GeForce RTX 2060
[codecarbon 